In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import activations
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
%matplotlib inline
from matplotlib import pyplot as plt
from keras.callbacks import CSVLogger
import gc


config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

root = os.getcwd()


In [2]:
def data_generator(encoded_images_array, ground_truth_array, start_index):
    
    path_post_enc = "C:\\Users\\Furkan\\Desktop\\BLG4901E\\nii_deney\\post_enc_frames"
    path_pre_enc = "C:\\Users\\Furkan\\Desktop\\BLG4901E\\nii_deney\\pre_enc_frames"
    batch_size = 1   # bir klasör kadar (130) görsel verir
    
    folders = os.listdir(path_post_enc)
    for i in range(start_index, start_index + batch_size):
        os.chdir(os.path.join(path_post_enc, folders[i]))
        files = os.listdir(os.getcwd())
        for file in files:
            with open(file, 'rb') as yuv_file:
                encoded_images_array.append(np.frombuffer(yuv_file.read(256 * 256), dtype=np.uint8).reshape((256, 256)))

    folders = os.listdir(path_pre_enc)
    for i in range(start_index, start_index + batch_size):
        os.chdir(os.path.join(path_pre_enc, folders[i]))
        files = os.listdir(os.getcwd())
        for file in files:
            with open(file, 'rb') as yuv_file:
                ground_truth_array.append(np.frombuffer(yuv_file.read(256 * 256), dtype=np.uint8).reshape((256, 256)))

    return start_index + batch_size


In [3]:
def validation_data(X_val, y_val):
    path_post_enc = "C:\\Users\\Furkan\\Desktop\\BLG4901E\\nii_deney\\post_enc_frames"
    path_pre_enc = "C:\\Users\\Furkan\\Desktop\\BLG4901E\\nii_deney\\pre_enc_frames"
    
    folders = os.listdir(path_post_enc)
    for i in range(570, 578):
        os.chdir(os.path.join(path_post_enc, folders[i]))
        files = os.listdir(os.getcwd())
        for file in files:
            with open(file, 'rb') as yuv_file:
                X_val.append(np.frombuffer(yuv_file.read(256 * 256), dtype=np.uint8).reshape((256, 256)))

    folders = os.listdir(path_pre_enc)
    for i in range(570, 578):
        os.chdir(os.path.join(path_pre_enc, folders[i]))
        files = os.listdir(os.getcwd())
        for file in files:
            with open(file, 'rb') as yuv_file:
                y_val.append(np.frombuffer(yuv_file.read(256 * 256), dtype=np.uint8).reshape((256, 256)))

In [4]:
'''
def SpatialAttention(input):
    x = layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(input)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(x)
    out = layers.Activation('sigmoid')(x)

    return out
''' 
def SpatialAttention(input):
    x_shortcut = input
    x = layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(input)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('sigmoid')(x)
    out = layers.Multiply()([x_shortcut, x])

    return out

In [5]:

def ChannelAttention(input):
    sh = input.get_shape().as_list()
    x = layers.GlobalAveragePooling2D()(input)
    x = layers.Dense(8)(x)
    x = layers.ReLU()(x)
    x = layers.Dense(sh[-1])(x)
    out = layers.Activation('sigmoid')(x)

    return out


In [6]:
def ResidualAttentionBlock(x):
    x_shortcut = x

    x = layers.Conv2D(filters=1, kernel_size=3, padding="same")(x)  # conv 1
   # x = layers.BatchNormalization()(x)
    spatial_attention = SpatialAttention(x)  # spatial attention
    x = layers.Multiply()([x, spatial_attention])  # multiply

    x = layers.Conv2D(filters=64, kernel_size=3, padding="same")(x)  # conv 2
   # x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.1)(x)  # leaky relu
    x = layers.Conv2D(filters=1, kernel_size=3, padding="same")(x)  # conv 2
  #  x = layers.BatchNormalization()(x)

    x = layers.Add()([x, x_shortcut])  # add
  #  x = layers.BatchNormalization()(x)
  #  x = activations.sigmoid(x)

    return x


In [7]:
def CNN(num_blocks):
    input_layer = layers.Input(shape=(256, 256, 1))
    x = input_layer

    for _ in range(num_blocks):
        x = ResidualAttentionBlock(x)

    model = tf.keras.Model(inputs=input_layer, outputs=x)
    return model


In [8]:
num_blocks = 30
Model = CNN(num_blocks)

In [9]:
'''
X_val = []
y_val = []
validation_data(X_val, y_val)
X_val = np.array(X_val) / 255.0
y_val = np.array(y_val) / 255.0
'''

'\nX_val = []\ny_val = []\nvalidation_data(X_val, y_val)\nX_val = np.array(X_val) / 255.0\ny_val = np.array(y_val) / 255.0\n'

In [10]:
csv_logger = CSVLogger('C:\\Users\\Furkan\\Desktop\\BLG4901E\\CNN\\log.csv', append=True, separator=';')

In [11]:
checkpoint_filepath = 'C:\\Users\\Furkan\\Desktop\\BLG4901E\\CNN\\chkpt\\model_checkpoint.h5'

In [12]:
checkpoint_filepath = 'C:\\Users\\Furkan\\Desktop\\BLG4901E\\CNN\\chkpt\\model_checkpoint.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='loss',
    mode='max',
    save_best_only=False,
    save_weights_only=False,
    save_freq="epoch")

In [13]:
# Model = tf.keras.models.load_model(checkpoint_filepath)

In [14]:
optimizer = optimizers.Adam(learning_rate=1e-4, beta_1=0, beta_2=0.999)
Model.compile(optimizer=optimizer, loss='mae', metrics=["mse", "mae"])
# Model.summary()

In [ ]:
# epochs = 460 (0.8 test size)
epochs = 460
for i in range (epochs):
    file_path = 'C:\\Users\\Furkan\\Desktop\\BLG4901E\\CNN\\devam_indisi.txt'
    with open(file_path, 'r') as file:
        st_ix = int(file.read())
        
    X_train = []
    y_train = []
    st_ix = data_generator(X_train, y_train, st_ix)

    if st_ix == 1:
        with open('C:\\Users\\Furkan\\Desktop\\BLG4901E\\CNN\\kac_epoch.txt', 'r+') as file:
            a = int(file.read())
            a += 1
            file.seek(0)
            file.truncate()
            file.write(f"{a}")
        st_ix = 0
    
    X_train = np.array(X_train) / 255.0
    y_train = np.array(y_train) / 255.0

    Model.fit(
        x=X_train,
        y=y_train,
        epochs=1,
        batch_size=1,
        shuffle=True,
        callbacks=[model_checkpoint_callback, csv_logger]
    )

    gc.collect()

    with open(file_path, 'w') as file:
        file.write(f"{st_ix}")

 26/130 [=====>........................] - ETA: 26s - loss: 0.0155 - mse: 0.0010 - mae: 0.0155

In [ ]:
'''
predictions = Model.predict(X_val)
predictions[predictions < 0] = 0
predictions[predictions > 255] = 255
'''

In [ ]:
'''
idx = 0
cv2.imwrite("C:\\Users\\Furkan\\Desktop\\BLG4901E\\CNN\\gorseller\\X.png", X_val[idx] * 255.0)
cv2.imwrite("C:\\Users\\Furkan\\Desktop\\BLG4901E\\CNN\\gorseller\\pred.png", predictions[idx] * 255.0)
cv2.imwrite("C:\\Users\\Furkan\\Desktop\\BLG4901E\\CNN\\gorseller\\y.png", y_val[idx] * 255.0)
'''